# Breadthwise state discovering algorithm
## Bucket state search problem

In [1]:
from dataclasses import dataclass
import numpy as np
import queue
from collections import defaultdict
import logging

l = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [17]:
# Pour the bucket to 0, raise error if action is forbidden 
def rule_empty(state: np.array, index):
    tmp = state.copy()
    if tmp[index] == 0:
        raise RuntimeError('is empty')
    tmp[index] = 0

    l.debug(f'empty: {state}[{index}] -> {tmp}[{index}]')

    return tmp

# Fill the bucket to max, raise error if action is forbidden 
def rule_full(state: np.array, index, vls):
    tmp = state.copy()
    if tmp[index] == vls[index]:
        raise RuntimeError('is full')
    tmp[index] = vls[index]

    l.debug(f'full: {state}[{index}] -> {tmp}[{index}]')

    return tmp

# Pour from i_from bucket to i_to bucket, raise error if action is forbidden 
def rule_transpose(state: np.array, i_from, i_to, vls):
    tmp = state.copy()

    if tmp[i_from] == 0:
        raise RuntimeError('from is empty')

    if tmp[i_to] == vls[i_to]:
        raise RuntimeError('to is full')

    avail_vl = vls[i_to] - tmp[i_to]
    if avail_vl <= tmp[i_from]:
        tmp[i_to] = vls[i_to]
        tmp[i_from] = tmp[i_from] - avail_vl
    else:
        tmp[i_to] += tmp[i_from]
        tmp[i_from] = 0

    l.debug(
        f'transpose: {state} f[{i_from}] t[{i_to}] -> {tmp} f[{i_from}] t[{i_to}]')

    return tmp

# Generate all possible combinations using rules
def expand(state, vls) -> list[np.array]:
    l.info(f'expanding {state}')

    inc = []
    try:
        inc.append(rule_empty(state, 0))
        l.info('-> empty 0')
    except RuntimeError:
        pass

    try:
        inc.append(rule_empty(state, 1))
        l.info('-> empty 1')
    except RuntimeError:
        pass

    try:
        inc.append(rule_empty(state, 0))
        inc.append(rule_empty(state, 1))
        l.info('-> empty 0, 1')
    except RuntimeError:
        pass

    try:
        inc.append(rule_full(state, 0, vls))
        l.info('-> full 0')
    except RuntimeError:
        pass

    try:
        inc.append(rule_full(state, 1, vls))
        l.info('-> full 1')
    except RuntimeError:
        pass

    try:
        inc.append(rule_full(state, 0, vls))
        inc.append(rule_full(state, 1, vls))
        l.info('-> full 0, 1')
    except RuntimeError:
        pass

    try:
        inc.append(rule_empty(state, 0))
        inc.append(rule_full(state, 1, vls))
        l.info('-> empty 0, full 1')
    except RuntimeError:
        pass

    try:
        inc.append(rule_empty(state, 1))
        inc.append(rule_full(state, 0, vls))
        l.info('-> empty 1, full 0')
    except RuntimeError:
        pass

    try:
        inc.append(rule_transpose(state, 0, 1, vls))
        l.info('-> transpose 0 to 1')
    except RuntimeError:
        pass

    try:
        inc.append(rule_transpose(state, 1, 0, vls))
        l.info('-> transpose 1 to 0')
    except RuntimeError:
        pass

    l.info(f'done {state}: {len(inc)}: {inc}')

    return inc


# Test states for equality
def test_equal(a: np.array, b: np.array):
    return a[0] == b[0] and a[1] == b[1]


class N:  # Node to trace states (! states are traced fro target to start so no need to hold next node, only parent)
    parent = None
    payload = None

    def __init__(self, payload, parent=None):
        self.payload = payload
        self.parent = parent

    def backtrace(self, vls):
        l.info(f'^ {self.payload} | max {vls}')
        if self.parent is not None:
            self.parent.backtrace(vls)


### Configuration and algotithm start

In [20]:
max_volumes = np.array([5.0, 3.0])
start_volumes = np.array([0.0, 0.0])
end_volumes = np.array([3.0, 2.0])

l.info(f'max: {max_volumes}, start: {start_volumes} -> end: {end_volumes}')

root = N(start_volumes, None)

to_open = [root]
closed = []


# Loop over all to be opened states
while len(to_open):
    n = to_open.pop(0)
    closed.append(n)

    l.info(f'opening: {n.payload}')

    # Find all incident states
    inc = expand(n.payload, max_volumes)

    # Exclude expanded or to be expanded states
    for i_s in inc:
        # Test state is target state
        if test_equal(i_s, end_volumes):
            l.info('--- target state reached ---')
            i_n = N(i_s, n)
            i_n.backtrace(max_volumes)
            break

        # Not in to open list
        for o_n in to_open:
            if test_equal(i_s, o_n.payload):
                break
        else:
            # Not in closed list
            for c_n in closed:
                if test_equal(i_s, c_n.payload):
                    break
            else:
                l.info(f'+ discovered: {i_s}')
                # Create incident node from incident state to backtrack route
                i_n = N(i_s, n)
                to_open.append(i_n)
    else:  # Stop at closest solution
        continue

    break
else:
    l.info('--- target state unreachable ---')


INFO:__main__:max: [5. 3.], start: [0. 0.] -> end: [3. 2.]
INFO:__main__:opening: [0. 0.]
INFO:__main__:expanding [0. 0.]
INFO:__main__:-> full 0
INFO:__main__:-> full 1
INFO:__main__:-> full 0, 1
INFO:__main__:done [0. 0.]: 4: [array([5., 0.]), array([0., 3.]), array([5., 0.]), array([0., 3.])]
INFO:__main__:+ discovered: [5. 0.]
INFO:__main__:+ discovered: [0. 3.]
INFO:__main__:opening: [5. 0.]
INFO:__main__:expanding [5. 0.]
INFO:__main__:-> empty 0
INFO:__main__:-> full 1
INFO:__main__:-> empty 0, full 1
INFO:__main__:-> transpose 0 to 1
INFO:__main__:done [5. 0.]: 6: [array([0., 0.]), array([0., 0.]), array([5., 3.]), array([0., 0.]), array([5., 3.]), array([2., 3.])]
INFO:__main__:+ discovered: [5. 3.]
INFO:__main__:+ discovered: [2. 3.]
INFO:__main__:opening: [0. 3.]
INFO:__main__:expanding [0. 3.]
INFO:__main__:-> empty 1
INFO:__main__:-> full 0
INFO:__main__:-> empty 1, full 0
INFO:__main__:-> transpose 1 to 0
INFO:__main__:done [0. 3.]: 6: [array([0., 0.]), array([5., 3.]), a